In [17]:
from IPython.display import clear_output
import numpy as np

In [14]:
DATA_COUNT = 1000
DIM = 4
DATA_LIMIT = 1

Rotation stuffs

In [23]:
def rotate_vec(dim=4, theta=None, i=None, j=None):

    if theta is None:
        theta = 2 * np.pi * np.random.random()

    if i is None:
        i = np.random.choice(dim)

    if j is None:
        j = np.random.choice(dim)
        while i == j:
            j = np.random.choice(dim)

    rotational_matrix = np.identity(dim)
    rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
    rotational_matrix[i][j] = np.sin(theta)
    rotational_matrix[j][i] = -np.sin(theta)

    return rotational_matrix


def random_rot(dim, count=10):
    id = np.identity(dim)
    for i in range(count):
        id = id.dot(rotate_vec(dim=dim))
    return id


Creating data

In [36]:


def tensor_diff(a, b):
    c = a - b
    c = np.squeeze(c)
    c_t = np.var(c)
    c = np.square(c)
    c = np.sqrt(np.average(c))
    return np.average(c), c_t, np.exp(- np.average(c)) * 100

def dropout_p_matrix(percent,shp, step = True):
    dropout_p = np.random.random(shp)
    d_l = np.vectorize(lambda x : 1 if x < percent else 0)
    if step:
        return(d_l(dropout_p))
    else:
        return dropout_p

def hyp(vex):
    return vex

In [25]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [47]:

def regressutil(weights, training_data, training_label, epochs, testing_data, testing_label,alpha = 0.001, update_interval = 10, retain = 1):
    for epoch in range(epochs):
        predicted_label = training_data.dot(weights[0] * dropout_p_matrix(retain, weights[0].shape)).dot(weights[1]* dropout_p_matrix(retain, weights[1].shape))
        
        dW1 = - (2 * training_data.dot(weights[0]).T.dot(training_label - predicted_label)) / training_data.size
        dW0 = - (2 * training_data.dot(weights[1]).T.dot(training_label - predicted_label)) / training_data.size
        weights[0] -= dW0 * alpha
        weights[1] -= dW1 * alpha

        if epoch % (epochs // update_interval) == 0:
            clear_output()
            print("Error :", tensor_diff(testing_data.dot(weights[0]).dot(weights[1]), testing_label))
            print("Weight")
            print(weights)
    return weights

        

In [44]:
def regress(W, vecs, vecsy, epochs):
    vecs = np.array(np.split(vecs, 10))
    vecsy = np.array(np.split(vecsy, 10))
    for i in range(10):
        W = regressutil(W, np.vstack(np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0)), epochs //10, vecs[i], vecsy[i], update_interval = 1, retain = (i/30 + 0.7))

    return W

In [48]:
W = np.array([np.random.random((DIM, DIM)), np.random.random((DIM, DIM))])
vecs = (np.random.random((DATA_COUNT, DIM)) * 2 * DATA_LIMIT) - DATA_LIMIT
r = random_rot(DIM)
vecsy = hyp(vecs)


In [41]:
r

array([[ 0.4676, -0.7321, -0.4919,  0.0581],
       [ 0.3748, -0.3433,  0.8552, -0.1018],
       [-0.106 , -0.0774, -0.102 , -0.9861],
       [-0.7935, -0.5832,  0.1277,  0.1179]])

In [ ]:
regress(W, vecs, vecsy, 800000)

In [50]:
W = regressutil(W, vecs, vecsy, 80000, vecs, vecsy, update_interval = 10, retain = 1, alpha= 0.001)
W

Error : (0.000239809987958626, 5.7508177311833943e-08, 99.97602187641581)
Weight
[[[-0.5972 -0.646   0.3821  0.6189]
  [ 0.5563  0.4067  0.541   0.0547]
  [ 0.9732 -1.035   0.2639  0.7677]
  [ 1.0611  1.4822 -0.4655  0.6246]]

 [[-0.6451  0.2188  0.4989  0.0073]
  [ 0.1975  0.3743 -0.46    0.3365]
  [ 0.4195  1.3659 -0.1781 -0.3172]
  [ 0.9399 -0.2419  0.1112  0.5536]]]


array([[[-0.5974, -0.6462,  0.3821,  0.6186],
        [ 0.5563,  0.4066,  0.541 ,  0.0548],
        [ 0.9733, -1.035 ,  0.264 ,  0.7678],
        [ 1.0612,  1.4824, -0.4656,  0.6248]],

       [[-0.6453,  0.2186,  0.4988,  0.0071],
        [ 0.1975,  0.3745, -0.46  ,  0.3366],
        [ 0.4197,  1.3662, -0.1782, -0.3168],
        [ 0.9401, -0.2419,  0.1113,  0.5538]]])

In [51]:
W[0].dot(W[1])

array([[ 0.9998, -0.0002, -0.    , -0.0003],
       [-0.0001,  0.9997,  0.0001, -0.0003],
       [ 0.0001,  0.0001,  1.    ,  0.0001],
       [-0.    ,  0.    , -0.0001,  1.    ]])